In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import torch
import argparse
import torch.utils.model_zoo as model_zoo
from torch.nn.parameter import Parameter
import numpy as np
from datetime import datetime
import pickle as pkl

# imports
from wsddn import WSDDN
from voc_dataset import *
import wandb
from utils import nms, tensor_to_PIL
from PIL import Image, ImageDraw
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# hyper-parameters
# ------------
parser = argparse.ArgumentParser(description='PyTorch ImageNet Training')
parser.add_argument(
    '--lr',
    default=0.0001,
    type=float,
    help='Learning rate'
)
parser.add_argument(
    '--lr-decay-steps',
    default=150000,
    type=int,
    help='Interval at which the lr is decayed'
)
parser.add_argument(
    '--lr-decay',
    default=0.1,
    type=float,
    help='Decay rate of lr'
)
parser.add_argument(
    '--momentum',
    default=0.9,
    type=float,
    help='Momentum of optimizer'
)
parser.add_argument(
    '--weight-decay',
    default=0.0005,
    type=float,
    help='Weight decay'
)
parser.add_argument(
    '--epochs',
    default=5,
    type=int,
    help='Number of epochs'
)
parser.add_argument(
    '--val-interval',
    default=5000,
    type=int,
    help='Interval at which to perform validation'
)
parser.add_argument(
    '--disp-interval',
    default=10,
    type=int,
    help='Interval at which to perform visualization'
)
parser.add_argument(
    '--use-wandb',
    default=False,
    type=bool,
    help='Flag to enable visualization'
)

class args:
    parsed_args = parser.parse_known_args()[0]
    batch_size = 16
    workers = 2
    print_freq = parsed_args.disp_interval
    eval_freq= parsed_args.val_interval
    epochs = parsed_args.epochs
    lr= parsed_args.lr
    use_wandb = parsed_args.use_wandb
    pretrained = True
    momentum = parsed_args.momentum
    weight_decay = parsed_args.weight_decay
    #start_epoch = parsed_args.start_epoch
print(args.__dict__)

{'__module__': '__main__', 'parsed_args': Namespace(disp_interval=10, epochs=5, lr=0.0001, lr_decay=0.1, lr_decay_steps=150000, momentum=0.9, use_wandb=False, val_interval=5000, weight_decay=0.0005), 'batch_size': 16, 'workers': 2, 'print_freq': 10, 'eval_freq': 5000, 'epochs': 5, 'lr': 0.0001, 'use_wandb': False, 'pretrained': True, 'momentum': 0.9, 'weight_decay': 0.0005, '__dict__': <attribute '__dict__' of 'args' objects>, '__weakref__': <attribute '__weakref__' of 'args' objects>, '__doc__': None}


In [3]:
global args
# TODO (Q2.2): Load datasets and create dataloaders
dataset = VOCDataset('trainval', top_n=300, image_size=512, data_dir='./data/VOCdevkit/VOC2007/')
n = len(dataset)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(np.floor(n*0.8)), n-int(np.floor(n*0.8))])
train_sampler = torch.utils.data.SubsetRandomSampler(range(len(train_dataset)))

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.workers,
    pin_memory=True,
    sampler=train_sampler,
    collate_fn=custom_collate_fn_VOC,
    drop_last=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    shuffle=False,
    num_workers=args.workers,
    pin_memory=True,
    collate_fn=custom_collate_fn_VOC,
    drop_last=True)

# Initialize wandb logger
if args.use_wandb:
    wandb.init(project="vlr-hw1", reinit=True)

Path:/home/mymomo119966_mm/hw/hw1-object_localization/data/VOCdevkit/VOC2007


In [4]:
# Set random seed
rand_seed = 1024
if rand_seed is not None:
    np.random.seed(rand_seed)
    torch.manual_seed(rand_seed)

# Set output directory
output_dir = "./"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


def calculate_map():
    """
    Calculate the mAP for classification.
    """
    # TODO (Q2.3): Calculate mAP on test set.
    # Feel free to write necessary function parameters.
    pass


def test_model(model, val_loader=None, thresh=0.05):
    """
    Tests the networks and visualizes the detections
    :param thresh: Confidence threshold
    """
    with torch.no_grad():
        for iter, data in enumerate(val_loader):

            # one batch = data for one image
            image = data['image']
            target = data['label']
            wgt = data['wgt']
            rois = data['rois']
            gt_boxes = data['gt_boxes']
            gt_class_list = data['gt_classes']
            # TODO (Q2.3): perform forward pass, compute cls_probs
            
            
            # TODO (Q2.3): Iterate over each class (follow comments)
            for class_num in range(20):
                # get valid rois and cls_scores based on thresh
                
                # use NMS to get boxes and scores
                pass

            # TODO (Q2.3): visualize bounding box predictions when required
            calculate_map()


def train_model(model, train_loader=None, val_loader=None, optimizer=None, args=None):
    """
    Trains the network, runs evaluation and visualizes the detections
    """
    # Initialize training variables
    train_loss = 0
    step_cnt = 0
    for epoch in range(args.epochs):
        for i, data in enumerate(train_loader):
            debug=True if (epoch==0 and i==0) else False
            # TODO (Q2.2): get one batch and perform forward pass
            # one batch = data for one image
            image = data['image']
            target = data['label']
            wgt = data['wgt']
            rois = data['rois']
            gt_boxes = data['gt_boxes']
            gt_class_list = data['gt_classes']

            # TODO (Q2.2): perform forward pass
            # take care that proposal values should be in pixels
            # Convert inputs to cuda if training on GPU
            if debug: print(f"Input shapes: Image {image.size()}; ROIs:{len(rois), type(rois[0])}; targets:{target.size()}")
            cls_prob = model(image, rois, target)
            
            # backward pass and update
            loss = model.loss
            train_loss += loss.item()
            step_cnt += 1

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # TODO (Q2.2): evaluate the model every N iterations (N defined in handout)
            # Add wandb logging wherever necessary
            if iter % args.val_interval == 0 and iter != 0:
                model.eval()
                ap = test_model(model, val_loader)
                print("AP ", ap)
                model.train()
            
            # TODO (Q2.4): Perform all visualizations here
            # The intervals for different things are defined in the handout
            if i%args.disp_interval==0:
                pass
                #

    # TODO (Q2.4): Plot class-wise APs

<b> Main

In [5]:
# Create network and initialize
net = WSDDN(classes=dataset.CLASS_NAMES)

if os.path.exists('pretrained_alexnet.pkl'):
    pret_net = pkl.load(open('pretrained_alexnet.pkl', 'rb'))
else:
    pret_net = model_zoo.load_url(
        'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth')
    pkl.dump(pret_net,
    open('pretrained_alexnet.pkl', 'wb'), pkl.HIGHEST_PROTOCOL)
own_state = net.state_dict()

for name, param in pret_net.items():
    if name not in own_state:
        continue
    if isinstance(param, Parameter):
        param = param.data
    try:
        own_state[name].copy_(param)
    except:
        print('Did not find {}'.format(name))
        continue

# Move model to GPU and set train mode
net.load_state_dict(own_state)
net.cuda()
net.train()

Classes:['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']


/opt/conda/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


WSDDN(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=(1, 1), ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (roi_pool): RoIPool(output_size=(6, 6), spatial_scale=0.0625)
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=4096, out_features=4096, bias=True)
    (4): ReLU(inplace=True)
  

In [6]:
# TODO (Q2.2): Freeze AlexNet layers since we are loading a pretrained model
for param in net.features.parameters():
    param.requires_grad = False
# TODO (Q2.2): Create optimizer only for network parameters that are trainable
params = list(net.parameters())
optimizer = torch.optim.SGD(params, lr=args.lr, 
                            momentum=args.momentum, weight_decay=args.weight_decay)

In [14]:
# Training
train_model(net, train_loader, val_loader, optimizer, args)

Input shapes: Image torch.Size([16, 3, 512, 512]); ROIs:(16, <class 'torch.Tensor'>); targets:torch.Size([16, 20])
torch.Size([4800, 256, 6, 6])
torch.Size([4800, 9216])
torch.Size([4800, 20]) torch.Size([16, 20])
loss vec shape:torch.Size([20]), torch.Size([16, 20])


ValueError: Using a target size (torch.Size([16, 20])) that is different to the input size (torch.Size([20])) is deprecated. Please ensure they have the same size.